#Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
from numpy import random
import pprint

#reference to coursera course

#Data


*   GenData
*   DataPreprocessing



In [1]:
#gen 3-d data

def gen_data(n,filename):
    f = open(filename,"w")
    f.write("{} \n".format(n))
    for i in range(0,n):
        l_gen = []
        for j in range(0,3):
            g = round(random.uniform(0,100),3)
            l_gen.append(g)
        if i != n - 1:
            f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))
            f.write('\n')
        else:
            f.write("{} {} {}".format(l_gen[0],l_gen[1],l_gen[2]))
    f.close()

In [ ]:
#preprocess-data => convert to 3d vector for local hashing
points = []
with open('INP.txt','r',encoding='UTF-8') as f:
     n = int(f.readline())
     for i in range(n):
         line = f.readline()
         lst = re.split("\s+",line)
         if len(lst) >= 4:
            lst = lst[:-1]
         # -50.000 for move coordinate axis with O' is (50,50) => convenient for taking the space
         point = [round(float(x),3) - 50.000 for x in lst]
         point = np.array(point).reshape(1,3)
         points.append(point)
points = np.array(points)

#Helper Function

*   cosine_similarity
*   euclide_distance
*   knn
*   knn_euclide





In [ ]:
def cosine_similarity(a,b):
    dot = np.sum(a * b)
    cosin = dot/(np.linalg.norm(a,2) * np.linalg.norm(b,2))

    return cosin 

def euclide_distance(a,b):

    dist = 0.0
    for i in range(len(a)):
        dist += np.sum(pow((a[i] - b[i])/50.0,2))

    dist = np.sqrt(dist)
    return dist

def check(a,b):
    a = np.squeeze(a)
    b = np.squeeze(b)  
    for i in range(len(a)):
        if a[i] != b[i]:
           return False
    return True

def knn(v, v_neighbor_l, k=1):
    similarity_l = []

    for ele in v_neighbor_l:
        cos_similarity = cosine_similarity(v,ele)

        similarity_l.append(cos_similarity)
        
    sorted_idxs = np.argsort(similarity_l)
    
    k_idx = sorted_idxs[-k:]

    return k_idx

def knn_euclide(v, v_neighbor_l, k = 1):
    similarity_l = []

    for ele in v_neighbor_l:
        dist = euclide_distance(v,ele)

        similarity_l.append(dist)
    
    sorted_idxs = np.argsort(similarity_l)

    k_idx = sorted_idxs[-k:]

    return k_idx

#Class Locality Sensitive Hashing



In [ ]:
class LSH:
    '''
    class LSH:
      #attributes
        points <=> data
        N_VECS <=> number of elements in data
        N_DIMS <=> dimension of data
        n_vecs <=> number of elements in one bucket
        N_UNIVERSE <=> number of random hyperplane
        N_PLANES <=> number of planes

      #method
        @abstractmethod
        def __init__(self,points)

        #helper method
        def create_planes(self,n_vecs,N_UNIVERSE)
        def hash_v(self,v,planes)
        def create_hash_table(self,vecs,planes)
        def flipped_one_bits(self,hash)
        def flipped_two_bits(self,hash)

      #opition1: find point in data
        def check_point_in_data(self,point_query)
      
      #option2: find nearest point in data
        #main method for universe  
        def create_hash_tables(self)
        def find_k_nearest_neighbor_universe(self,point_query,k = 1,id_query = -1)

        #main method for flipped method
        def find_k_nearest_neighbor_flipped(self,point_query,k = 1,id_query = -1)

        #result for query
        def view_result(self,method,k = 3,id_query = -1,point_query = None)
        def compute_accuracy(self,method,k = 3,id_query = -1,point_query = None)
    '''
    def __init__(self,points):
       self.points = points
       self.N_VECS = len(points)
       self.N_DIMS = points[0].shape[1]

    def create_planes(self,n_vecs,N_UNIVERSE):
       self.n_vecs = n_vecs
       self.N_UNIVERSE = N_UNIVERSE
       self.N_PLANES = int(np.ceil(np.log2(self.N_VECS/n_vecs)))
       
       random.seed(0)
       planes_l = [np.random.normal(size = (self.N_DIMS,self.N_PLANES)) for _ in range(N_UNIVERSE)]
  
       self.planes_l = planes_l
     
    def hash_v(self,v,planes):
        dot_product = np.dot(v,planes) # (1,3) * (3,4)
        sign_of_dot_product = np.sign(dot_product)
        h = sign_of_dot_product >= 0
        #convert 2D to 1D array
        h = np.squeeze(h)
        hash_value = 0
        n_planes = planes.shape[1]
        
        for i in range(n_planes):
            hash_value += pow(2,i) * h[i]
        hash_value = int(hash_value)

        return hash_value

    def create_hash_table(self,vecs,planes):
        num_of_planes = planes.shape[1]
        num_of_buckets = pow(2,num_of_planes)

        #dictionary hash_table store value
        hash_table = {i : [] for i in range(num_of_buckets)}

        #dictionary id_table store id of value
        id_table = {i : [] for i in range(num_of_buckets)}

        for i,v in enumerate(vecs):

            h = self.hash_v(v,planes)

            #store v in "h" buckets
            hash_table[h].append(v)

            #store id of v in "h" id_buckets
            id_table[h].append(i)

        return hash_table,id_table

    def create_hash_tables(self):
        hash_tables = []
        id_tables = []

        for id in range(self.N_UNIVERSE):
            planes = self.planes_l[id]
            hash_table,id_table = self.create_hash_table(self.points,planes)
            hash_tables.append(hash_table)
            id_tables.append(id_table)    
      
        self.hash_tables = hash_tables
        self.id_tables = id_tables

    def find_k_nearest_neighbor_universe(self,point_query,k = 1,id_query = -1):
        points_neighbor_l = list()
        idxs_neighbor_l = list()
        idxs_neighbor_set = set() #check faster if point already exist in set

        for uni_id in range(self.N_UNIVERSE):
            planes = self.planes_l[uni_id]

            #get hash_value of point_query to find neareast neighbor of this
            hash_value = self.hash_v(point_query,planes)

            hash_table = self.hash_tables[uni_id]
            id_table = self.id_tables[uni_id]

            #get subset of hash_table to lookup
            new_ids_table_consider = id_table[hash_value]
            new_points_table_consider = hash_table[hash_value]
            
            #if id_query exist in hash_table
            if id_query != -1 and id_query in new_ids_table_consider:
              new_ids_table_consider.remove(id_query)

            for i,new_id in enumerate(new_ids_table_consider):
                if new_id not in idxs_neighbor_set:
                  point_i = new_points_table_consider[i]
                  
                  #add list to consider
                  points_neighbor_l.append(point_i)
                  idxs_neighbor_l.append(new_id)

                  #add set to faster checking
                  idxs_neighbor_set.add(new_id)

        print("Number of points to consider: ",len(points_neighbor_l))    
        points_neighbor_l_arr = np.array(points_neighbor_l)
        k_nearest_neighbor_idx = knn(point_query,points_neighbor_l_arr,k)
        k_nearest_neighbor_idx_l = [idxs_neighbor_l[idx] for idx in k_nearest_neighbor_idx]
        k_nearest_neighbor_idx_point = [self.points[idx] for idx in k_nearest_neighbor_idx_l]

        return k_nearest_neighbor_idx_l,k_nearest_neighbor_idx_point
    
    def flipped_one_bits(self,hash):
        bits = bin(hash).format(self.N_PLANES)[2:]

        hashs = list()
        
        for i in range(len(bits)):
            bits_temp = list(bits)  
            if bits[i] == '0':
              bits_temp[i] = '1'
            else:
              bits_temp[i] = '0'
            bits_temp = ''.join(bits_temp)
            hashs.append(int(bits_temp,2))
        return hashs

    def flipped_two_bits(self,hash):
        bits = bin(hash).format(self.N_PLANES)[2:]

        hashs = list()

        for i in range(len(bits)):
            for j in range(i + 1,len(bits)):
                bits_temp = list(bits)
                
                bits_temp[i] = chr(1 - (int(bits[i]) - 48))
                bits_temp[j] = chr(1 - (int(bits[j]) - 48))

                bits_temp = ''.join(bits_temp)
                hashs.append(int(bits_temp,2))
        return hashs
    
    def check_point_in_data(self,point_query):
        np.random.seed(1)
        plane = np.random.normal(size = (self.N_DIMS,self.N_PLANES))

        hash_tables,id_tables = self.create_hash_table(self.points,plane)
        hash_value = self.hash_v(point_query,plane)

        hash_table_subset = hash_tables[hash_value]
        id_table_subset = id_tables[hash_value]
        
        for i,id in enumerate(id_table_subset):
            if check(point_query,self.points[id]) is True:
               return id

        return -1

    def find_k_nearest_neighbor_flipped(self,point_query,k = 3,id_query = -1):
        np.random.seed(1)
        plane = np.random.normal(size = (self.N_DIMS,self.N_PLANES))

        points_neighbor_l = list()
        idxs_neighbor_l = list()
        idxs_neighbor_set = set()

        hash_tables,id_tables = self.create_hash_table(self.points,plane)
        hash_value = self.hash_v(point_query,plane)

        two_bits = self.flipped_two_bits(hash_value)
        one_bits = self.flipped_one_bits(hash_value)

        hashs = list()
        hashs.append(hash_value)

        hashs = hashs + one_bits + two_bits
        
        for hash in hashs:
            #subset of table
            hash_table_subset = hash_tables[hash]
            id_table_subset = id_tables[hash]
            
            if id_query != -1 and id_query in id_table_subset:
               id_table_subset.remove(id_query)

            for i,id in enumerate(id_table_subset):
                if id not in idxs_neighbor_set:
                  
                   point_i = hash_table_subset[i]

                   points_neighbor_l.append(point_i)
                   idxs_neighbor_l.append(id)

                   idxs_neighbor_set.add(id)

        print("Number of points to consider: ",len(points_neighbor_l))    
        points_neighbor_l_arr = np.array(points_neighbor_l)
        
        k_nearest_neighbor_idx = knn(point_query,points_neighbor_l_arr,k)
        k_nearest_neighbor_idx_l = [idxs_neighbor_l[idx] for idx in k_nearest_neighbor_idx]

        k_nearest_neighbor_idx_point = [self.points[idx] for idx in k_nearest_neighbor_idx_l]

        return k_nearest_neighbor_idx_l,k_nearest_neighbor_idx_point     

    def view_result(self,method,k = 3,id_query = -1,point_query = None):
        if id_query != -1:
           point_query = self.points[id_query]
        if method == "universe":
           id_l,point_l = self.find_k_nearest_neighbor_universe(point_query,k,id_query)
        elif method == "flipped":
           id_l,point_l = self.find_k_nearest_neighbor_flipped(point_query,k,id_query)
        else:
           print("error in method")
           raise ValueError("invalid method")

        point_normal = [ele + 50.00 for ele in point_l]
        print("data point to query: ",point_query + 50.00)
        print(f"{k} approximate nearest points:\n")
        #update method print: point and cosine_simalarity
        pprint.pprint(point_normal)

    def compute_accuracy(self,method,k = 3,id_query = -1,point_query = None):
        #implement time in python      
        if id_query != -1:
           point_query = self.points[id_query]
        
        k_l = knn(point_query,self.points,k)

        label_l = [self.points[ele] + 50.00 for ele in k_l]

        if method == "universe":
           id_l,point_l = self.find_k_nearest_neighbor_universe(point_query,k,id_query)
        elif method == "flipped":
           id_l,point_l = self.find_k_nearest_neighbor_flipped(point_query,k,id_query)
        else:
           print("error in method")
           raise ValueError("invalid method")
        
        predict_l = [ele + 50.00 for ele in point_l]

        count = 0.0
        for i in range(len(predict_l)):
            for j in range(i + 1,len(label_l)):
                if check(predict_l[i],label_l[j]) is True:            
                   count += 1
                   break

        acc = (count * 1.0) / len(predict_l)
        print("the probability of the exact nearest number of points found by the LHS method: {}%".format(round(acc * 100,3)))

#MAIN


*   Class Definition
*   Test point in data
*   Test random point
*   Check point in data





In [ ]:
print(LSH.__doc__)


    class LSH:
      #attributes
        points <=> data
        N_VECS <=> number of elements in data
        N_DIMS <=> dimension of data
        n_vecs <=> number of elements in one bucket
        N_UNIVERSE <=> number of random hyperplane
        N_PLANES <=> number of planes

      #method
        @abstractmethod
        def __init__(self,points)

        #helper method
        def create_planes(self,n_vecs,N_UNIVERSE)
        def hash_v(self,v,planes)
        def create_hash_table(self,vecs,planes)
        def flipped_one_bits(self,hash)
        def flipped_two_bits(self,hash)

      #opition1: find point in data
        def check_point_in_data(self,point_query)
      
      #option2: find nearest point in data
        #main method for universe  
        def create_hash_tables(self)
        def find_k_nearest_neighbor_universe(self,point_query,k = 1,id_query = -1)

        #main method for flipped method
        def find_k_nearest_neighbor_flipped(self,point_query,k = 1,id_query

Test point in data

In [ ]:
model = LSH(points)

#method for universe:
model.create_planes(10,25)
model.create_hash_tables()
print("===================================================")
print("METHOD UNIVERSE")
model.view_result("universe",3,2)
print("===================================================")
print("METHOD FLIPPED")
model.view_result("flipped",3,2)
print("===================================================")

METHOD UNIVERSE
Number of points to consider:  467
data point to query:  [[77.172 26.55  91.224]]
3 approximate nearest points:

[array([[34.253,  9.604, 51.126]]),
 array([[66.712, 52.352, 83.005]]),
 array([[75.009, 38.075, 70.908]])]
METHOD FLIPPED
Number of points to consider:  337
data point to query:  [[77.172 26.55  91.224]]
3 approximate nearest points:

[array([[86.916, 12.277, 20.187]]),
 array([[82.154, 20.884, 45.036]]),
 array([[75.009, 38.075, 70.908]])]


In [ ]:
model.compute_accuracy("universe",500,50)

Number of points to consider:  444
the probability of the exact nearest number of points found by the LHS method: 60.586%


Test random point

In [ ]:
def gen_random_point(dims):
    l_gen = []

    for j in range(0,dims):
        g = round(random.uniform(0,100),3)
        l_gen.append(g)
    point = [round(float(x),3) - 50.000 for x in l_gen]
    point = np.array(point).reshape(1,3)
    return point

#preprocess data
point = gen_random_point(3)

In [ ]:
print("===================================================")
print("METHOD UNIVERSE")
model.view_result("universe",3,-1,point)
print("===================================================")
print("METHOD FLIPPED")
model.view_result("flipped",3,-1,point)
print("===================================================")

METHOD UNIVERSE
Number of points to consider:  302
data point to query:  [[68.65  83.463  1.829]]
3 approximate nearest points:

[array([[71.519, 78.119,  5.912]]),
 array([[57.084, 61.602, 29.722]]),
 array([[52.178, 55.412, 42.334]])]
METHOD FLIPPED
Number of points to consider:  204
data point to query:  [[68.65  83.463  1.829]]
3 approximate nearest points:

[array([[71.519, 78.119,  5.912]]),
 array([[57.084, 61.602, 29.722]]),
 array([[52.178, 55.412, 42.334]])]


In [ ]:
model.compute_accuracy("universe",400,-1,point)

Number of points to consider:  302
the probability of the exact nearest number of points found by the LHS method: 98.344%


Check point in data

In [ ]:
model.check_point_in_data(point)

-1